In [1]:
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import cna
import multianndata as mad

# For Aparna's Import
import scipy.io

# For reproducibility
np.random.seed(0)  

In [3]:
sc_uni = dict()

In [4]:
sc_cond = dict()

In [5]:
sc_chron = dict()

In [6]:
def run_cna(variable, cna_obj, covs = None):
    if covs is not None:
        cna_res = cna.tl._association.association(cna_obj, #dataset 
                                                  cna_obj.samplem[variable], #phenotype
            #                                       batches=d.samplem.processing_batch, #batches
                                                  covs = cna_obj.samplem[covs],
                                                  Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                                  ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                            #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                                 )
    else:
        cna_res = cna.tl._association.association(cna_obj, #dataset 
                                                  cna_obj.samplem[variable], #phenotype
            #                                       batches=d.samplem.processing_batch, #batches
                                                 # covs = d.samplem[covs]
                                                  Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                                  ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                            #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                                 )
        
    return(cna_res)
def cna_test(meta_df, harmony_df, umap_df, variable, covars = None):
    print(meta_df.shape)
    print(harmony_df.shape)
    print(umap_df.shape)
    cna_object = mad.MultiAnnData(X=harmony_df, obs=meta_df, sampleid="sample")
    if covars is not None:   
        cna_object.obs_to_sample(covars + [variable])
        umap_df.index = cna_object.obs.index
        cna_object.obsm['X_umap'] = umap_df
        cna.pp.knn(cna_object)
        np.random.seed(0) 
        final_res = run_cna(variable, cna_object, covs = covars)
    else:
        cna_object.obs_to_sample([variable])
        umap_df.index = cna_object.obs.index
        cna_object.obsm['X_umap'] = umap_df
        cna.pp.knn(cna_object)
        np.random.seed(0) 
        final_res = run_cna(variable, cna_object, covs = None)
    print(final_res.k)
    print(final_res.ks)
    print('p =', final_res.p, ',', final_res.k, 'PCs used')
    print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(final_res.k, final_res.r2))
    return(final_res)

# CASE/CONTROL

## Single Cell

In [6]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/sc_meta.csv')
print(meta.shape)

(159761, 41)


In [7]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/sc_harmony.csv')
print(harmony.shape)

(159761, 20)


In [8]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/sc_umap.csv')
print(umap.shape)

(159761, 2)


In [9]:
sc_res = cna_test(meta_df = meta, harmony_df = harmony, 
                  umap_df = umap, variable = 'Case.Control', covars = None)

(159761, 41)
(159761, 20)
(159761, 2)


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies
computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 62.75668716430664
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 40.95512008666992
	20th percentile R2(t,t-1): 0.7591327071189881
	taking step 3
	median kurtosis: 27.98756217956543
	20th percentile R2(t,t-1): 0.9079185128211975
	taking step 4
	median kurtosis: 19.238365173339844
	20th percentile R2(t,t-1): 0.9430045485496521
	taking step 5
	median kurtosis: 13.708256721496582
	20th percentile R2(t,t-1): 0.9602678775787353
	taking step 6
	median kurtosis: 10.32024097442627
	20th percentile R2(t,t-1): 0.9722456693649292
	taking step 7
	median kurtosis: 8.258467674255371
	20th percentile R2(t,t-1): 0.9815566897392273
stopping after 7 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:74: UserWarning: global association p-value attained minimal possible value. Consider increasing Nnull
  warnings.warn('global association p-value attained minimal possible value. '+\


computing neighborhood-level FDRs
18
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 9.999000099990002e-05 , 18 PCs used
total r^2 between top 18 NAM PCs and outcome is 0.67


In [11]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/sc_ncorr.csv", 
               sc_res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/sc_fdrs.csv", 
               sc_res.fdrs, delimiter=",")

## Single Nuclear

In [12]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/sn_meta.csv')
print(meta.shape)

(52464, 35)


In [13]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/sn_harmony.csv')
print(harmony.shape)

(52464, 20)


In [14]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/sn_umap.csv')
print(umap.shape)

(52464, 2)


In [15]:
sn_res = cna_test(meta_df = meta, harmony_df = harmony, 
                  umap_df = umap, variable = 'Case.Control', covars = None)

(52464, 35)
(52464, 20)
(52464, 2)
['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies
computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 17.345572471618652
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 11.76611328125
	20th percentile R2(t,t-1): 0.7671453475952148
	taking step 3
	median kurtosis: 8.575805187225342
	20th percentile R2(t,t-1): 0.9140462040901184
	taking step 4
	median kurtosis: 6.701585054397583
	20th percentile R2(t,t-1): 0.9498215079307556
stopping after 4 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing neighborhood-level FDRs
11
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.00029997000299970003 , 11 PCs used
total r^2 between top 11 NAM PCs and outcome is 0.84


In [16]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/sn_ncorr.csv", 
               sn_res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/sn_fdrs.csv", 
               sn_res.fdrs, delimiter=",")

## HIGH CHRONICITY

### SINGLE CELL

In [57]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/high_chronicity_sc_meta.csv')
print(meta.shape)

(36469, 34)


In [58]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/high_chronicity_sc_harmony.csv')
print(harmony.shape)

(36469, 20)


In [59]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/high_chronicity_sc_umap.csv')
print(umap.shape)

(36469, 2)


In [60]:
sc_res = cna_test(meta_df = meta, harmony_df = harmony, 
                  umap_df = umap, variable = 'Case.Control', covars = None)

(36469, 34)
(36469, 20)
(36469, 2)
['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies
computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 22.792888641357422
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 14.906187057495117
	20th percentile R2(t,t-1): 0.7668335437774658
	taking step 3
	median kurtosis: 10.756307601928711
	20th percentile R2(t,t-1): 0.9143049120903015
	taking step 4
	median kurtosis: 8.079665184020996
	20th percentile R2(t,t-1): 0.9487549066543579
stopping after 4 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:74: UserWarning: global association p-value attained minimal possible value. Consider increasing Nnull
  warnings.warn('global association p-value attained minimal possible value. '+\


computing neighborhood-level FDRs
18
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 9.999000099990002e-05 , 18 PCs used
total r^2 between top 18 NAM PCs and outcome is 0.64


In [61]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/high_chronicity_sc_ncorr.csv", 
               sc_res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/high_chronicity_sc_fdrs.csv", 
               sc_res.fdrs, delimiter=",")

### SINGLE NUCLEAR

In [62]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/high_chronicity_sn_meta.csv')
print(meta.shape)

(14803, 31)


In [63]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/high_chronicity_sn_harmony.csv')
print(harmony.shape)

(14803, 20)


In [64]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/high_chronicity_sn_umap.csv')
print(umap.shape)

(14803, 2)


In [65]:
sn_res = cna_test(meta_df = meta, harmony_df = harmony, 
                  umap_df = umap, variable = 'Case.Control', covars = None)

(14803, 31)
(14803, 20)
(14803, 2)
['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies
computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 7.521926558361265
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 5.028265408539582
	20th percentile R2(t,t-1): 0.7688755393028259
	taking step 3
	median kurtosis: 3.909364196362083
	20th percentile R2(t,t-1): 0.9133266806602478
stopping after 3 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:57: UserWarning: data supported use of 20 NAM PCs, which is the maximum considered. Consider allowing more PCs by using the "ks" argument.
  warnings.warn(('data supported use of {} NAM PCs, which is the maximum considered. '+\
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:74: UserWarning: global association p-value attained minimal possible value. Consider increasing Nnull
  warnings.warn('global association p-value attained minimal possible value. '+\


computing neighborhood-level FDRs
20
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 9.999000099990002e-05 , 20 PCs used
total r^2 between top 20 NAM PCs and outcome is 1.00


In [66]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/high_chronicity_sn_ncorr.csv", 
               sn_res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/high_chronicity_sn_fdrs.csv", 
               sn_res.fdrs, delimiter=",")

## HIGH ACTIVITY

### SINGLE CELL

In [27]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/high_activity_sc_meta.csv')
print(meta.shape)

(53275, 35)


In [28]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/high_activity_sc_harmony.csv')
print(harmony.shape)

(53275, 20)


In [29]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/high_activity_sc_umap.csv')
print(umap.shape)

(53275, 2)


In [30]:
sc_res = cna_test(meta_df = meta, harmony_df = harmony, 
                  umap_df = umap, variable = 'Case.Control', covars = None)

(53275, 35)
(53275, 20)
(53275, 2)
['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies
computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 26.352561950683594
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 16.908029556274414
	20th percentile R2(t,t-1): 0.7583823442459107
	taking step 3
	median kurtosis: 12.290385246276855
	20th percentile R2(t,t-1): 0.9123182415962219
	taking step 4
	median kurtosis: 9.092205047607422
	20th percentile R2(t,t-1): 0.948067057132721
	taking step 5
	median kurtosis: 7.0531511306762695
	20th percentile R2(t,t-1): 0.9642295479774475
stopping after 5 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:74: UserWarning: global association p-value attained minimal possible value. Consider increasing Nnull
  warnings.warn('global association p-value attained minimal possible value. '+\


computing neighborhood-level FDRs
16
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 9.999000099990002e-05 , 16 PCs used
total r^2 between top 16 NAM PCs and outcome is 0.65


In [32]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/high_activity_sc_ncorr.csv", 
               sc_res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/high_activity_sc_fdrs.csv", 
               sc_res.fdrs, delimiter=",")

### SINGLE NUCLEAR

In [33]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/high_activity_sn_meta.csv')
print(meta.shape)

(18093, 29)


In [34]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/high_activity_sn_harmony.csv')
print(harmony.shape)

(18093, 20)


In [35]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/high_activity_sn_umap.csv')
print(umap.shape)

(18093, 2)


In [36]:
sn_res = cna_test(meta_df = meta, harmony_df = harmony, 
                  umap_df = umap, variable = 'Case.Control', covars = None)

(18093, 29)
(18093, 20)
(18093, 2)
['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies
computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 8.195440739871747
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 5.744108865188026
	20th percentile R2(t,t-1): 0.7654945254325867
	taking step 3
	median kurtosis: 4.5927370182013805
	20th percentile R2(t,t-1): 0.9134642004966735
stopping after 3 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing neighborhood-level FDRs
4
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.046495350464953504 , 4 PCs used
total r^2 between top 4 NAM PCs and outcome is 0.60


In [38]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/high_activity_sn_ncorr.csv", 
               sn_res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/high_activity_sn_fdrs.csv", 
               sn_res.fdrs, delimiter=",")

## CLASS V

### SINGLE CELL

In [47]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/class_V_sc_meta.csv')
print(meta.shape)

(48054, 32)


In [48]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/class_V_sc_harmony.csv')
print(harmony.shape)

(48054, 20)


In [49]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/class_V_sc_umap.csv')
print(umap.shape)

(48054, 2)


In [50]:
sc_res = cna_test(meta_df = meta, harmony_df = harmony, 
                  umap_df = umap, variable = 'Case.Control', covars = None)

(48054, 32)
(48054, 20)
(48054, 2)
['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies
computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 26.54051399230957
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 16.82704448699951
	20th percentile R2(t,t-1): 0.760974109172821
	taking step 3
	median kurtosis: 12.074318885803223
	20th percentile R2(t,t-1): 0.9105615019798279
	taking step 4
	median kurtosis: 8.651598930358887
	20th percentile R2(t,t-1): 0.9468539953231812
	taking step 5
	median kurtosis: 6.538233041763306
	20th percentile R2(t,t-1): 0.9641885757446289
stopping after 5 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:74: UserWarning: global association p-value attained minimal possible value. Consider increasing Nnull
  warnings.warn('global association p-value attained minimal possible value. '+\


computing neighborhood-level FDRs
12
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 9.999000099990002e-05 , 12 PCs used
total r^2 between top 12 NAM PCs and outcome is 0.65


In [51]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/class_V_sc_ncorr.csv", 
               sc_res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/class_V_sc_fdrs.csv", 
               sc_res.fdrs, delimiter=",")

### SINGLE NUCLEAR

In [52]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/class_V_sn_meta.csv')
print(meta.shape)

(10596, 24)


In [53]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/class_V_sn_harmony.csv')
print(harmony.shape)

(10596, 20)


In [54]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/class_V_sn_umap.csv')
print(umap.shape)

(10596, 2)


In [55]:
sn_res = cna_test(meta_df = meta, harmony_df = harmony, 
                  umap_df = umap, variable = 'Case.Control', covars = None)

(10596, 24)
(10596, 20)
(10596, 2)
['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies
computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 5.967700572323677
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 4.097410575814481
	20th percentile R2(t,t-1): 0.7853367328643799
	taking step 3
	median kurtosis: 3.218924273802589
	20th percentile R2(t,t-1): 0.9293646812438965
stopping after 3 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:74: UserWarning: global association p-value attained minimal possible value. Consider increasing Nnull
  warnings.warn('global association p-value attained minimal possible value. '+\


computing neighborhood-level FDRs
15
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 9.999000099990002e-05 , 15 PCs used
total r^2 between top 15 NAM PCs and outcome is 1.00


In [56]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/class_V_sn_ncorr.csv", 
               sn_res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/case_control/class_V_sn_fdrs.csv", 
               sn_res.fdrs, delimiter=",")

# AGE

## Single Cell

In [3]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/age_sex_response_race/sc_meta_age.csv')
print(meta.shape)

(154032, 39)


In [4]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/age_sex_response_race/sc_harmony_age.csv')
print(harmony.shape)

(154032, 20)


In [5]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/age_sex_response_race/sc_umap_age.csv')
print(umap.shape)

(154032, 2)


In [6]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[A][B]', 'Race_[B]', 'Race_[B][AI]', 'Race_[B][W]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 'Final_Site_Cincinnati',
       'Final_Site_Einstein', 'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 'Final_Site_UCLA',
       'Final_Site_UCSD', 'Final_Site_UCSF'])
d.samplem.head()

/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


['cell' 'sample' 'cluster_names' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[A][B],Race_[B],...,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCLA,Final_Site_UCSD,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0142,1.0,1.489979,4.0,16.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0366,2.0,-1.068691,7.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
AMPSLEkid_cells_0966,1.0,-0.794548,5.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
AMPSLEkid_cells_2748,1.0,-1.068691,8.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0138,1.0,0.576169,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [7]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [8]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Age, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 50.24106979370117
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 33.02307891845703
	20th percentile R2(t,t-1): 0.7615886211395264
	taking step 3
	median kurtosis: 22.02227783203125
	20th percentile R2(t,t-1): 0.9084737420082092
	taking step 4
	median kurtosis: 14.879904747009277
	20th percentile R2(t,t-1): 0.9440323233604431
	taking step 5
	median kurtosis: 10.545235633850098
	20th percentile R2(t,t-1): 0.9626616954803466
	taking step 6
	median kurtosis: 7.996293544769287
	20th percentile R2(t,t-1): 0.9743828177452087
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:57: UserWarning: data supported use of 20 NAM PCs, which is the maximum considered. Consider allowing more PCs by using the "ks" argument.
  warnings.warn(('data supported use of {} NAM PCs, which is the maximum considered. '+\


computing neighborhood-level FDRs
20
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.043795620437956206 , 20 PCs used
total r^2 between top 20 NAM PCs and outcome is 0.22


In [10]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/ALL_TISSUE/age_sex_response_race/sc_age_ncorr.csv", 
               res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/ALL_TISSUE/age_sex_response_race/sc_age_fdrs.csv", 
               res.fdrs, delimiter=",")

In [11]:
sc_uni['Age'] = res.p

## Single Nuclei

In [26]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/age_sex_response_race/sn_meta_age.csv')
print(meta.shape)

(50173, 9)


In [27]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/age_sex_response_race/sn_harmony_age.csv')
print(harmony.shape)

(50173, 20)


In [28]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/age_sex_response_race/sn_umap_age.csv')
print(umap.shape)

(50173, 2)


In [29]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Age'])
d.samplem.head()

['cell' 'Sex' 'sample' 'cluster_names' 'Responder.Status' 'Race' 'ISN'
 'Type']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


,Age
sample,
AMPSLEkid_cells_0137,10.0
AMPSLEkid_cells_0138,16.0
AMPSLEkid_cells_0139,20.0
AMPSLEkid_cells_0147,20.0
AMPSLEkid_cells_0366,3.0


In [30]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [31]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [32]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Age, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 13.259119033813477
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 9.410001754760742
	20th percentile R2(t,t-1): 0.7799340009689331
	taking step 3
	median kurtosis: 6.884187698364258
	20th percentile R2(t,t-1): 0.9179095029830933
stopping after 3 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:57: UserWarning: data supported use of 20 NAM PCs, which is the maximum considered. Consider allowing more PCs by using the "ks" argument.
  warnings.warn(('data supported use of {} NAM PCs, which is the maximum considered. '+\


computing neighborhood-level FDRs
20
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.1740825917408259 , 20 PCs used
total r^2 between top 20 NAM PCs and outcome is 0.63


In [33]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/ALL_TISSUE/age_sex_response_race/sn_age_ncorr.csv", 
               res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/ALL_TISSUE/age_sex_response_race/sn_age_fdrs.csv", 
               res.fdrs, delimiter=",")

# SEX

## Single Cell

In [12]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/age_sex_response_race/sc_meta_sex.csv')
print(meta.shape)

(154032, 39)


In [13]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/age_sex_response_race/sc_harmony_sex.csv')
print(harmony.shape)

(154032, 20)


In [14]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/age_sex_response_race/sc_umap_sex.csv')
print(umap.shape)

(154032, 2)


In [15]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[A][B]', 'Race_[B]', 'Race_[B][AI]', 'Race_[B][W]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 'Final_Site_Cincinnati',
       'Final_Site_Einstein', 'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 'Final_Site_UCLA',
       'Final_Site_UCSD', 'Final_Site_UCSF'])
d.samplem.head()

/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


['cell' 'sample' 'cluster_names' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[A][B],Race_[B],...,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCLA,Final_Site_UCSD,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0142,1.0,1.489979,4.0,16.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0366,2.0,-1.068691,7.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
AMPSLEkid_cells_0966,1.0,-0.794548,5.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
AMPSLEkid_cells_2748,1.0,-1.068691,8.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0138,1.0,0.576169,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [16]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [17]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [18]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Sex, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 50.24106979370117
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 33.02307891845703
	20th percentile R2(t,t-1): 0.7615886211395264
	taking step 3
	median kurtosis: 22.02227783203125
	20th percentile R2(t,t-1): 0.9084737420082092
	taking step 4
	median kurtosis: 14.879904747009277
	20th percentile R2(t,t-1): 0.9440323233604431
	taking step 5
	median kurtosis: 10.545235633850098
	20th percentile R2(t,t-1): 0.9626616954803466
	taking step 6
	median kurtosis: 7.996293544769287
	20th percentile R2(t,t-1): 0.9743828177452087
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
9
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.1571842815718428 , 9 PCs used
total r^2 between top 9 NAM PCs and outcome is 0.12


In [19]:
sc_uni['Sex'] = res.p

# RESPONSE

## Single Cell

In [20]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/age_sex_response_race/sc_meta_response.csv')
print(meta.shape)

(130904, 36)


In [21]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/age_sex_response_race/sc_harmony_response.csv')
print(harmony.shape)

(130904, 20)


In [22]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/age_sex_response_race/sc_umap_response.csv')
print(umap.shape)

(130904, 2)


In [26]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[A][B]', 'Race_[B]', 'Race_[B][AI]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 
       'Final_Site_Einstein', 'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 
       'Final_Site_UCSD', 'Final_Site_UCSF'])
d.samplem.head()

/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


['cell' 'sample' 'cluster_names' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[A][B],Race_[B],...,Final_Site_Einstein,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCSD,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0142,1.0,1.612695,4.0,16.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
AMPSLEkid_cells_0366,2.0,-1.030751,7.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
AMPSLEkid_cells_0966,1.0,-0.747524,5.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
AMPSLEkid_cells_2748,1.0,-1.030751,8.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0138,1.0,0.668607,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [27]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [28]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [29]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Responder_Status, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 40.874427795410156
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 26.85399627685547
	20th percentile R2(t,t-1): 0.7663337349891662
	taking step 3
	median kurtosis: 18.085410118103027
	20th percentile R2(t,t-1): 0.9100850939750671
	taking step 4
	median kurtosis: 12.471662521362305
	20th percentile R2(t,t-1): 0.9460514187812805
	taking step 5
	median kurtosis: 9.115138053894043
	20th percentile R2(t,t-1): 0.9641231536865235
	taking step 6
	median kurtosis: 7.136253356933594
	20th percentile R2(t,t-1): 0.9752503514289856
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.15308469153084692 , 1 PCs used
total r^2 between top 1 NAM PCs and outcome is 0.04


In [30]:
sc_uni['Responder_Status'] = res.p

## Single Nuclear

In [81]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/age_sex_response_race/sn_meta_response.csv')
print(meta.shape)

(49390, 32)


In [82]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/age_sex_response_race/sn_harmony_response.csv')
print(harmony.shape)

(49390, 20)


In [83]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/age_sex_response_race/sn_umap_response.csv')
print(umap.shape)

(49390, 2)


In [88]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
      'Race_[B]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 
       'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 
       'Final_Site_UCSF'])
d.samplem.head()

/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


['cell' 'sample' 'cluster_names' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[B],Race_[U],...,Final_ISN_[IV][V],Final_ISN_[V],Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0137,1.0,0.096672,3.0,5.0,0.0,1.0,2.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
AMPSLEkid_cells_0138,1.0,0.670324,0.0,0.0,1.0,1.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
AMPSLEkid_cells_0139,1.0,1.052759,3.0,2.0,0.0,1.0,2.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
AMPSLEkid_cells_0147,1.0,1.052759,9.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
AMPSLEkid_cells_0366,2.0,-1.050634,7.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [89]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [90]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [91]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Responder_Status, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 12.916019439697266
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 9.15547227859497
	20th percentile R2(t,t-1): 0.7803142070770264
	taking step 3
	median kurtosis: 6.726650714874268
	20th percentile R2(t,t-1): 0.9174365162849426
stopping after 3 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:57: UserWarning: data supported use of 20 NAM PCs, which is the maximum considered. Consider allowing more PCs by using the "ks" argument.
  warnings.warn(('data supported use of {} NAM PCs, which is the maximum considered. '+\


computing neighborhood-level FDRs
20
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.2746725327467253 , 20 PCs used
total r^2 between top 20 NAM PCs and outcome is 0.60


# RACE

## Single Cell

In [3]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/age_sex_response_race/sc_meta_race.csv')
print(meta.shape)

(154032, 39)


In [4]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/age_sex_response_race/sc_harmony_race.csv')
print(harmony.shape)

(154032, 20)


In [5]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/age_sex_response_race/sc_umap_race.csv')
print(umap.shape)

(154032, 2)


In [6]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[A][B]', 'Race_[B]', 'Race_[B][AI]', 'Race_[B][W]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 'Final_Site_Cincinnati',
       'Final_Site_Einstein', 'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 'Final_Site_UCLA',
       'Final_Site_UCSD', 'Final_Site_UCSF'])
d.samplem.head()

/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


['cell' 'sample' 'cluster_names' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[A][B],Race_[B],...,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCLA,Final_Site_UCSD,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0142,1.0,1.489979,4.0,16.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0366,2.0,-1.068691,7.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
AMPSLEkid_cells_0966,1.0,-0.794548,5.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
AMPSLEkid_cells_2748,1.0,-1.068691,8.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0138,1.0,0.576169,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [7]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [8]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [56]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[A]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 50.24106979370117
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 33.02307891845703
	20th percentile R2(t,t-1): 0.7615886211395264
	taking step 3
	median kurtosis: 22.02227783203125
	20th percentile R2(t,t-1): 0.9084737420082092
	taking step 4
	median kurtosis: 14.879904747009277
	20th percentile R2(t,t-1): 0.9440323233604431
	taking step 5
	median kurtosis: 10.545235633850098
	20th percentile R2(t,t-1): 0.9626616954803466
	taking step 6
	median kurtosis: 7.996293544769287
	20th percentile R2(t,t-1): 0.9743828177452087
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
18
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.09499050094990501 , 18 PCs used
total r^2 between top 18 NAM PCs and outcome is 0.19


In [57]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[A][B]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
6
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.36196380361963804 , 6 PCs used
total r^2 between top 6 NAM PCs and outcome is 0.06


In [95]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[B]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 50.24106979370117
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 33.02307891845703
	20th percentile R2(t,t-1): 0.7615886211395264
	taking step 3
	median kurtosis: 22.02227783203125
	20th percentile R2(t,t-1): 0.9084737420082092
	taking step 4
	median kurtosis: 14.879904747009277
	20th percentile R2(t,t-1): 0.9440323233604431
	taking step 5
	median kurtosis: 10.545235633850098
	20th percentile R2(t,t-1): 0.9626616954803466
	taking step 6
	median kurtosis: 7.996293544769287
	20th percentile R2(t,t-1): 0.9743828177452087
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
16
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.049795020497950204 , 16 PCs used
total r^2 between top 16 NAM PCs and outcome is 0.19


In [97]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/age_sex_response_race/sc_race_[B]_ncorr.csv", 
               res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/age_sex_response_race/sc_race_[B]_fdrs.csv", 
               res.fdrs, delimiter=",")

In [59]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[B][AI]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
6
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.33556644335566443 , 6 PCs used
total r^2 between top 6 NAM PCs and outcome is 0.06


In [60]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[B][W]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.8569143085691431 , 1 PCs used
total r^2 between top 1 NAM PCs and outcome is 0.00


In [61]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[U]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:57: UserWarning: data supported use of 20 NAM PCs, which is the maximum considered. Consider allowing more PCs by using the "ks" argument.
  warnings.warn(('data supported use of {} NAM PCs, which is the maximum considered. '+\


computing neighborhood-level FDRs
20
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.16598340165983402 , 20 PCs used
total r^2 between top 20 NAM PCs and outcome is 0.20


In [37]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[W]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 50.24106979370117
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 33.02307891845703
	20th percentile R2(t,t-1): 0.7615886211395264
	taking step 3
	median kurtosis: 22.02227783203125
	20th percentile R2(t,t-1): 0.9084737420082092
	taking step 4
	median kurtosis: 14.879904747009277
	20th percentile R2(t,t-1): 0.9440323233604431
	taking step 5
	median kurtosis: 10.545235633850098
	20th percentile R2(t,t-1): 0.9626616954803466
	taking step 6
	median kurtosis: 7.996293544769287
	20th percentile R2(t,t-1): 0.9743828177452087
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
3
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.0337966203379662 , 3 PCs used
total r^2 between top 3 NAM PCs and outcome is 0.08


In [63]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/age_sex_response_race/sc_race_[W]_ncorr.csv", 
               res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/age_sex_response_race/sc_race_[W]_fdrs.csv", 
               res.fdrs, delimiter=",")

In [38]:
sc_uni['Race'] = res.p

In [43]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[W]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      covs = d.samplem[['Final_Activity', 
                                                        'First_biop', 
                                                        'Final_ISN_[III][V]', 'Final_ISN_[IV]',
                                                        'Final_ISN_[IV][V]', 'Final_ISN_[V]',
                                                        'Final_Site_Einstein', 'Final_Site_JHU', 
                                                        'Final_Site_MUSC', 'Final_Site_NYU',
                                                        'Final_Site_Rochester', 'Final_Site_Texas Tech', 
                                                        'Final_Site_UCSD', 'Final_Site_UCSF']], 
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 50.24106979370117
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 33.02307891845703
	20th percentile R2(t,t-1): 0.7615886211395264
	taking step 3
	median kurtosis: 22.02227783203125
	20th percentile R2(t,t-1): 0.9084737420082092
	taking step 4
	median kurtosis: 14.879904747009277
	20th percentile R2(t,t-1): 0.9440323233604431
	taking step 5
	median kurtosis: 10.545235633850098
	20th percentile R2(t,t-1): 0.9626616954803466
	taking step 6
	median kurtosis: 7.996293544769287
	20th percentile R2(t,t-1): 0.9743828177452087
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:57: UserWarning: data supported use of 20 NAM PCs, which is the maximum considered. Consider allowing more PCs by using the "ks" argument.
  warnings.warn(('data supported use of {} NAM PCs, which is the maximum considered. '+\


computing neighborhood-level FDRs
20
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.026697330266973303 , 20 PCs used
total r^2 between top 20 NAM PCs and outcome is 0.26


In [44]:
sc_cond['Race'] = res.p

In [9]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[W]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      covs = d.samplem[['Final_Activity', 'Final_Chronicity',
                                                        'First_biop', 
                                                        'Final_ISN_[III][V]', 'Final_ISN_[IV]',
                                                        'Final_ISN_[IV][V]', 'Final_ISN_[V]',
                                                        'Final_Site_Einstein', 'Final_Site_JHU', 
                                                        'Final_Site_MUSC', 'Final_Site_NYU',
                                                        'Final_Site_Rochester', 'Final_Site_Texas Tech', 
                                                        'Final_Site_UCSD', 'Final_Site_UCSF']], 
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 50.24106979370117
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 33.02307891845703
	20th percentile R2(t,t-1): 0.7615886211395264
	taking step 3
	median kurtosis: 22.02227783203125
	20th percentile R2(t,t-1): 0.9084737420082092
	taking step 4
	median kurtosis: 14.879904747009277
	20th percentile R2(t,t-1): 0.9440323233604431
	taking step 5
	median kurtosis: 10.545235633850098
	20th percentile R2(t,t-1): 0.9626616954803466
	taking step 6
	median kurtosis: 7.996293544769287
	20th percentile R2(t,t-1): 0.9743828177452087
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:57: UserWarning: data supported use of 20 NAM PCs, which is the maximum considered. Consider allowing more PCs by using the "ks" argument.
  warnings.warn(('data supported use of {} NAM PCs, which is the maximum considered. '+\


computing neighborhood-level FDRs
20
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.047295270472952705 , 20 PCs used
total r^2 between top 20 NAM PCs and outcome is 0.25


In [11]:
sc_chron['Race'] = res.p

## Single Nuclei

In [71]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/age_sex_response_race/sn_meta_race.csv')
print(meta.shape)

(50173, 33)


In [72]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/age_sex_response_race/sn_harmony_race.csv')
print(harmony.shape)

(50173, 20)


In [73]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/age_sex_response_race/sn_umap_race.csv')
print(umap.shape)

(50173, 2)


In [74]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Race_[A]', 'Race_[B]', 
                 'Race_[U]', 'Race_[W]'])
d.samplem.head()

['cell' 'sample' 'cluster_names' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


,Race_[A],Race_[B],Race_[U],Race_[W]
sample,,,,
AMPSLEkid_cells_0137,0.0,1.0,0.0,0.0
AMPSLEkid_cells_0138,0.0,0.0,1.0,0.0
AMPSLEkid_cells_0139,0.0,1.0,0.0,0.0
AMPSLEkid_cells_0147,0.0,0.0,1.0,0.0
AMPSLEkid_cells_0366,0.0,0.0,0.0,1.0


In [75]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [76]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [77]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[A]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 13.259119033813477
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 9.410001754760742
	20th percentile R2(t,t-1): 0.7799340009689331
	taking step 3
	median kurtosis: 6.884187698364258
	20th percentile R2(t,t-1): 0.9179095029830933
stopping after 3 steps
covariate-adjusted NAM not found; computing and saving
computing SVD


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


performing association test
computing neighborhood-level FDRs
2
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.49975002499750026 , 2 PCs used
total r^2 between top 2 NAM PCs and outcome is 0.16


In [78]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[B]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
17
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.0160983901609839 , 17 PCs used
total r^2 between top 17 NAM PCs and outcome is 0.69


In [88]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/age_sex_response_race/sn_race_[B]_ncorr.csv", 
               res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/age_sex_response_race/sn_race_[B]_fdrs.csv", 
               res.fdrs, delimiter=",")

In [79]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[U]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:57: UserWarning: data supported use of 20 NAM PCs, which is the maximum considered. Consider allowing more PCs by using the "ks" argument.
  warnings.warn(('data supported use of {} NAM PCs, which is the maximum considered. '+\


computing neighborhood-level FDRs
20
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.6727327267273273 , 20 PCs used
total r^2 between top 20 NAM PCs and outcome is 0.49


In [80]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[W]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
4
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.0165983401659834 , 4 PCs used
total r^2 between top 4 NAM PCs and outcome is 0.43


In [86]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/age_sex_response_race/sn_race_[W]_ncorr.csv", 
               res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/age_sex_response_race/sn_race_[W]_fdrs.csv", 
               res.fdrs, delimiter=",")

# SITE

## Single Cell

In [12]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/first_biop_pred_use_site/sc_meta_site.csv')
print(meta.shape)

(154032, 39)


In [13]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/first_biop_pred_use_site/sc_harmony_site.csv')
print(harmony.shape)

(154032, 20)


In [14]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/first_biop_pred_use_site/sc_umap_site.csv')
print(umap.shape)

(154032, 2)


In [15]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[A][B]', 'Race_[B]', 'Race_[B][AI]', 'Race_[B][W]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 'Final_Site_Cincinnati',
       'Final_Site_Einstein', 'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 'Final_Site_UCLA',
       'Final_Site_UCSD', 'Final_Site_UCSF'])
d.samplem.head()

/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


['cell' 'sample' 'cluster_names' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[A][B],Race_[B],...,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCLA,Final_Site_UCSD,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0142,1.0,1.489979,4.0,16.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0366,2.0,-1.068691,7.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
AMPSLEkid_cells_0966,1.0,-0.794548,5.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
AMPSLEkid_cells_2748,1.0,-1.068691,8.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0138,1.0,0.576169,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [16]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [17]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [46]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_Cincinnati, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 50.24106979370117
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 33.02307891845703
	20th percentile R2(t,t-1): 0.7615886211395264
	taking step 3
	median kurtosis: 22.02227783203125
	20th percentile R2(t,t-1): 0.9084737420082092
	taking step 4
	median kurtosis: 14.879904747009277
	20th percentile R2(t,t-1): 0.9440323233604431
	taking step 5
	median kurtosis: 10.545235633850098
	20th percentile R2(t,t-1): 0.9626616954803466
	taking step 6
	median kurtosis: 7.996293544769287
	20th percentile R2(t,t-1): 0.9743828177452087
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.2126787321267873 , 1 PCs used
total r^2 between top 1 NAM PCs and outcome is 0.03


In [59]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_Einstein, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
18
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.00029997000299970003 , 18 PCs used
total r^2 between top 18 NAM PCs and outcome is 0.52


In [60]:
sc_uni['Site'] = res.p

In [76]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_Einstein, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      covs = d.samplem[['Final_Chronicity', 'Final_Activity', 
                                                        'First_biop', 'Responder_Status', 'Race_[A]',
                                                        'Race_[A][B]', 'Race_[B]', 
                                                        'Race_[B][AI]', 'Race_[U]',
                                                        'Race_[W]', 
                                                        'Final_ISN_[III][V]', 'Final_ISN_[IV]',
                                                        'Final_ISN_[IV][V]', 'Final_ISN_[V]']], 
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 50.24106979370117
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 33.02307891845703
	20th percentile R2(t,t-1): 0.7615886211395264
	taking step 3
	median kurtosis: 22.02227783203125
	20th percentile R2(t,t-1): 0.9084737420082092
	taking step 4
	median kurtosis: 14.879904747009277
	20th percentile R2(t,t-1): 0.9440323233604431
	taking step 5
	median kurtosis: 10.545235633850098
	20th percentile R2(t,t-1): 0.9626616954803466
	taking step 6
	median kurtosis: 7.996293544769287
	20th percentile R2(t,t-1): 0.9743828177452087
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:57: UserWarning: data supported use of 20 NAM PCs, which is the maximum considered. Consider allowing more PCs by using the "ks" argument.
  warnings.warn(('data supported use of {} NAM PCs, which is the maximum considered. '+\


computing neighborhood-level FDRs
20
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.0033996600339966003 , 20 PCs used
total r^2 between top 20 NAM PCs and outcome is 0.51


In [79]:
sc_cond['Site'] = res.p

In [48]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_JHU, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
18
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.0725927407259274 , 18 PCs used
total r^2 between top 18 NAM PCs and outcome is 0.19


In [49]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_Michigan, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
3
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.7486251374862514 , 3 PCs used
total r^2 between top 3 NAM PCs and outcome is 0.02


In [50]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_MUSC, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.33236676332366766 , 1 PCs used
total r^2 between top 1 NAM PCs and outcome is 0.02


In [51]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_Northwell, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.855014498550145 , 1 PCs used
total r^2 between top 1 NAM PCs and outcome is 0.00


In [52]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_NYU, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
18
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.06429357064293571 , 18 PCs used
total r^2 between top 18 NAM PCs and outcome is 0.19


In [53]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_Rochester, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
5
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.5088491150884912 , 5 PCs used
total r^2 between top 5 NAM PCs and outcome is 0.05


In [55]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Final_Site_Texas Tech'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.23847615238476152 , 1 PCs used
total r^2 between top 1 NAM PCs and outcome is 0.03


In [56]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_UCLA, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.19488051194880512 , 1 PCs used
total r^2 between top 1 NAM PCs and outcome is 0.03


In [ ]:
sc_uni

In [57]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_UCSD, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.6020397960203979 , 1 PCs used
total r^2 between top 1 NAM PCs and outcome is 0.01


In [58]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_UCSF, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.8698130186981302 , 1 PCs used
total r^2 between top 1 NAM PCs and outcome is 0.00


## Single Nuclear

In [43]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/first_biop_pred_use_site/sn_meta_site.csv')
print(meta.shape)

(50173, 33)


In [44]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/first_biop_pred_use_site/sn_harmony_site.csv')
print(harmony.shape)

(50173, 20)


In [45]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/first_biop_pred_use_site/sn_umap_site.csv')
print(umap.shape)

(50173, 2)


In [50]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[B]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 'Final_Site_Cincinnati',
       'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 
       'Final_Site_UCSF'])
d.samplem.head()

/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


['cell' 'sample' 'cluster_names' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[B],Race_[U],...,Final_ISN_[V],Final_Site_Cincinnati,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0137,1.0,0.091298,3.0,5.0,0.0,1.0,2.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
AMPSLEkid_cells_0138,1.0,0.668818,0.0,0.0,1.0,1.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
AMPSLEkid_cells_0139,1.0,1.053831,3.0,2.0,0.0,1.0,2.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
AMPSLEkid_cells_0147,1.0,1.053831,9.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
AMPSLEkid_cells_0366,2.0,-1.063742,7.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [51]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [52]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [53]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_Cincinnati, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 13.259119033813477
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 9.410001754760742
	20th percentile R2(t,t-1): 0.7799340009689331
	taking step 3
	median kurtosis: 6.884187698364258
	20th percentile R2(t,t-1): 0.9179095029830933
stopping after 3 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing neighborhood-level FDRs
17
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.32456754324567544 , 17 PCs used
total r^2 between top 17 NAM PCs and outcome is 0.59


In [54]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_JHU, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:57: UserWarning: data supported use of 20 NAM PCs, which is the maximum considered. Consider allowing more PCs by using the "ks" argument.
  warnings.warn(('data supported use of {} NAM PCs, which is the maximum considered. '+\


computing neighborhood-level FDRs
20
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.37336266373362664 , 20 PCs used
total r^2 between top 20 NAM PCs and outcome is 0.58


In [55]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_Michigan, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
2
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.9051094890510949 , 2 PCs used
total r^2 between top 2 NAM PCs and outcome is 0.04


In [56]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_MUSC, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
15
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.202979702029797 , 15 PCs used
total r^2 between top 15 NAM PCs and outcome is 0.96


In [57]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_Northwell, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
15
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.5567443255674432 , 15 PCs used
total r^2 between top 15 NAM PCs and outcome is 0.45


In [58]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_NYU, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
20
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.73002699730027 , 20 PCs used
total r^2 between top 20 NAM PCs and outcome is 0.48


In [59]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_Rochester, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
17
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.7104289571042895 , 17 PCs used
total r^2 between top 17 NAM PCs and outcome is 0.43


In [62]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Final_Site_Texas Tech'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
14
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.19438056194380562 , 14 PCs used
total r^2 between top 14 NAM PCs and outcome is 0.99


In [63]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_UCSF, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
20
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.56994300569943 , 20 PCs used
total r^2 between top 20 NAM PCs and outcome is 0.50


# FIRST BIOPSY

## Single Cell

In [53]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/first_biop_pred_use_site/sc_meta_firstbiop.csv')
print(meta.shape)

(154032, 39)


In [54]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/first_biop_pred_use_site/sc_harmony_firstbiop.csv')
print(harmony.shape)

(154032, 20)


In [55]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/first_biop_pred_use_site/sc_umap_site.csv')
print(umap.shape)

(154032, 2)


In [56]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[A][B]', 'Race_[B]', 'Race_[B][AI]', 'Race_[B][W]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 'Final_Site_Cincinnati',
       'Final_Site_Einstein', 'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 'Final_Site_UCLA',
       'Final_Site_UCSD', 'Final_Site_UCSF'])
d.samplem.head()

/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


['cell' 'sample' 'cluster_names' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[A][B],Race_[B],...,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCLA,Final_Site_UCSD,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0142,1.0,1.489979,4.0,16.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0366,2.0,-1.068691,7.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
AMPSLEkid_cells_0966,1.0,-0.794548,5.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
AMPSLEkid_cells_2748,1.0,-1.068691,8.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0138,1.0,0.576169,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [57]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [58]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [67]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.First_biop, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 50.24106979370117
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 33.02307891845703
	20th percentile R2(t,t-1): 0.7615886211395264
	taking step 3
	median kurtosis: 22.02227783203125
	20th percentile R2(t,t-1): 0.9084737420082092
	taking step 4
	median kurtosis: 14.879904747009277
	20th percentile R2(t,t-1): 0.9440323233604431
	taking step 5
	median kurtosis: 10.545235633850098
	20th percentile R2(t,t-1): 0.9626616954803466
	taking step 6
	median kurtosis: 7.996293544769287
	20th percentile R2(t,t-1): 0.9743828177452087
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
16
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.0005999400059994001 , 16 PCs used
total r^2 between top 16 NAM PCs and outcome is 0.27


In [68]:
sc_uni['First_biopsy'] = res.p

In [59]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.First_biop, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      covs = d.samplem[['Final_Chronicity', 'Responder_Status',
                                                        'Final_ISN_[III][V]', 'Final_ISN_[IV]',
                                                        'Final_ISN_[IV][V]', 'Final_ISN_[V]',
                                                        'Race_[A]',
                                                        'Race_[A][B]', 'Race_[B]', 
                                                        'Race_[B][AI]', 'Race_[U]',
                                                        'Race_[W]', 
                                                        'Final_Site_Einstein', 'Final_Site_JHU', 
                                                        'Final_Site_MUSC', 'Final_Site_NYU',
                                                        'Final_Site_Rochester', 'Final_Site_Texas Tech', 
                                                        'Final_Site_UCSD', 'Final_Site_UCSF']],
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 50.24106979370117
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 33.02307891845703
	20th percentile R2(t,t-1): 0.7615886211395264
	taking step 3
	median kurtosis: 22.02227783203125
	20th percentile R2(t,t-1): 0.9084737420082092
	taking step 4
	median kurtosis: 14.879904747009277
	20th percentile R2(t,t-1): 0.9440323233604431
	taking step 5
	median kurtosis: 10.545235633850098
	20th percentile R2(t,t-1): 0.9626616954803466
	taking step 6
	median kurtosis: 7.996293544769287
	20th percentile R2(t,t-1): 0.9743828177452087
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:57: UserWarning: data supported use of 20 NAM PCs, which is the maximum considered. Consider allowing more PCs by using the "ks" argument.
  warnings.warn(('data supported use of {} NAM PCs, which is the maximum considered. '+\


computing neighborhood-level FDRs
20
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.0184981501849815 , 20 PCs used
total r^2 between top 20 NAM PCs and outcome is 0.36


In [61]:
sc_cond['First_biopsy'] = res.p

## Single Nuclear

In [64]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/first_biop_pred_use_site/sn_meta_firstbiop.csv')
print(meta.shape)

(50173, 33)


In [65]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/first_biop_pred_use_site/sn_harmony_firstbiop.csv')
print(harmony.shape)

(50173, 20)


In [66]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/first_biop_pred_use_site/sn_umap_firstbiop.csv')
print(umap.shape)

(50173, 2)


In [67]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[B]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 'Final_Site_Cincinnati',
       'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 
       'Final_Site_UCSF'])
d.samplem.head()

/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


['cell' 'sample' 'cluster_names' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[B],Race_[U],...,Final_ISN_[V],Final_Site_Cincinnati,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0137,1.0,0.091298,3.0,5.0,0.0,1.0,2.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
AMPSLEkid_cells_0138,1.0,0.668818,0.0,0.0,1.0,1.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
AMPSLEkid_cells_0139,1.0,1.053831,3.0,2.0,0.0,1.0,2.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
AMPSLEkid_cells_0147,1.0,1.053831,9.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
AMPSLEkid_cells_0366,2.0,-1.063742,7.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [68]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [69]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [70]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.First_biop, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 13.259119033813477
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 9.410001754760742
	20th percentile R2(t,t-1): 0.7799340009689331
	taking step 3
	median kurtosis: 6.884187698364258
	20th percentile R2(t,t-1): 0.9179095029830933
stopping after 3 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing neighborhood-level FDRs
17
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.0006999300069993001 , 17 PCs used
total r^2 between top 17 NAM PCs and outcome is 0.80


# PREDNISONE USE

## Single Cell

In [69]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/first_biop_pred_use_site/sc_meta_preduse.csv')
print(meta.shape)

(154032, 39)


In [70]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/first_biop_pred_use_site/sc_harmony_preduse.csv')
print(harmony.shape)

(154032, 20)


In [71]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/first_biop_pred_use_site/sc_umap_preduse.csv')
print(umap.shape)

(154032, 2)


In [72]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[A][B]', 'Race_[B]', 'Race_[B][AI]', 'Race_[B][W]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 'Final_Site_Cincinnati',
       'Final_Site_Einstein', 'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 'Final_Site_UCLA',
       'Final_Site_UCSD', 'Final_Site_UCSF'])
d.samplem.head()

/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


['cell' 'sample' 'cluster_names' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[A][B],Race_[B],...,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCLA,Final_Site_UCSD,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0142,1.0,1.489979,4.0,16.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0366,2.0,-1.068691,7.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
AMPSLEkid_cells_0966,1.0,-0.794548,5.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
AMPSLEkid_cells_2748,1.0,-1.068691,8.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0138,1.0,0.576169,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [73]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [74]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [75]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Pred_use, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 50.24106979370117
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 33.02307891845703
	20th percentile R2(t,t-1): 0.7615886211395264
	taking step 3
	median kurtosis: 22.02227783203125
	20th percentile R2(t,t-1): 0.9084737420082092
	taking step 4
	median kurtosis: 14.879904747009277
	20th percentile R2(t,t-1): 0.9440323233604431
	taking step 5
	median kurtosis: 10.545235633850098
	20th percentile R2(t,t-1): 0.9626616954803466
	taking step 6
	median kurtosis: 7.996293544769287
	20th percentile R2(t,t-1): 0.9743828177452087
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
3
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.45955404459554045 , 3 PCs used
total r^2 between top 3 NAM PCs and outcome is 0.04


In [77]:
sc_uni['Prednisone_use'] = res.p

## Single Nuclear

In [92]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/first_biop_pred_use_site/sn_meta_preduse.csv')
print(meta.shape)

(50173, 33)


In [93]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/first_biop_pred_use_site/sn_harmony_preduse.csv')
print(harmony.shape)

(50173, 20)


In [94]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/first_biop_pred_use_site/sn_umap_preduse.csv')
print(umap.shape)

(50173, 2)


In [101]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[B]',   'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 'Final_Site_Cincinnati',
        'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 
        'Final_Site_UCSF'])
d.samplem.head()

/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


['cell' 'sample' 'cluster_names' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[B],Race_[U],...,Final_ISN_[V],Final_Site_Cincinnati,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0137,1.0,0.091298,3.0,5.0,0.0,1.0,2.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
AMPSLEkid_cells_0138,1.0,0.668818,0.0,0.0,1.0,1.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
AMPSLEkid_cells_0139,1.0,1.053831,3.0,2.0,0.0,1.0,2.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
AMPSLEkid_cells_0147,1.0,1.053831,9.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
AMPSLEkid_cells_0366,2.0,-1.063742,7.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [102]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [103]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [104]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Pred_use, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 13.259119033813477
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 9.410001754760742
	20th percentile R2(t,t-1): 0.7799340009689331
	taking step 3
	median kurtosis: 6.884187698364258
	20th percentile R2(t,t-1): 0.9179095029830933
stopping after 3 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing neighborhood-level FDRs
13
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.507949205079492 , 13 PCs used
total r^2 between top 13 NAM PCs and outcome is 0.43


# CHRONICITY

## Single Cell

In [9]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/chronicity/sc_meta.csv')
print(meta.shape)

(140034, 39)


In [11]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/chronicity/sc_harmony.csv')
print(harmony.shape)

(140034, 20)


In [12]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/chronicity/sc_umap.csv')
print(umap.shape)

(140034, 2)


In [13]:
res = cna_test(meta,harmony, umap, 'Final_Chronicity', covars = ['First_biop', 'Responder_Status'])

(140034, 39)
(140034, 20)
(140034, 2)
['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 46.349185943603516
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 30.39740753173828
	20th percentile R2(t,t-1): 0.7639257311820984
	taking step 3
	median kurtosis: 20.410869598388672
	20th percentile R2(t,t-1): 0.908935010433197
	taking step 4
	median kurtosis: 13.953994750976562
	20th percentile R2(t,t-1): 0.9450386166572571
	taking step 5
	median kurtosis: 9.980635643005371
	20th percentile R2(t,t-1): 0.963342547416687
	taking step 6
	median kurtosis: 7.617109298706055
	20th percentile R2(t,t-1): 0.9739270806312561
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:57: UserWarning: data supported use of 20 NAM PCs, which is the maximum considered. Consider allowing more PCs by using the "ks" argument.
  warnings.warn(('data supported use of {} NAM PCs, which is the maximum considered. '+\
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:74: UserWarning: global association p-value attained minimal possible value. Consider increasing Nnull
  warnings.warn('global association p-value attained minimal possible value. '+\


computing neighborhood-level FDRs
20
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 9.999000099990002e-05 , 20 PCs used
total r^2 between top 20 NAM PCs and outcome is 0.64


In [14]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/chronicity/sc_conditional_ncorr.csv", 
               res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/chronicity/sc_conditional_fdrs.csv", 
               res.fdrs, delimiter=",")

## Single Nuclear

In [15]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/chronicity/sn_meta.csv')
print(meta.shape)

(40416, 32)


In [16]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/chronicity/sn_harmony.csv')
print(harmony.shape)

(40416, 20)


In [17]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/chronicity/sn_umap.csv')
print(umap.shape)

(40416, 2)


In [18]:
res = cna_test(meta,harmony, umap, 'Final_Chronicity', covars = ['First_biop', 'Responder_Status'])

(40416, 32)
(40416, 20)
(40416, 2)
['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies
computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 11.842386245727539
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 8.491312503814697
	20th percentile R2(t,t-1): 0.786330497264862
	taking step 3
	median kurtosis: 6.386176347732544
	20th percentile R2(t,t-1): 0.9200137615203857
stopping after 3 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing neighborhood-level FDRs
12
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.004999500049995001 , 12 PCs used
total r^2 between top 12 NAM PCs and outcome is 0.77


In [20]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/chronicity/sn_conditional_ncorr.csv", 
               res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/chronicity/sn_conditional_fdrs.csv", 
               res.fdrs, delimiter=",")

# ACTIVITY

## Single Cell

In [7]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/activity/sc_meta.csv')
print(meta.shape)

(140034, 39)


In [9]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/activity/sc_harmony.csv')
print(harmony.shape)

(140034, 20)


In [10]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/activity/sc_umap.csv')
print(umap.shape)

(140034, 2)


In [11]:
res = cna_test(meta, harmony, umap, 'Final_Activity')

(140034, 39)
(140034, 20)
(140034, 2)
['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies
computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 46.349185943603516
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 30.39740753173828
	20th percentile R2(t,t-1): 0.7639257311820984
	taking step 3
	median kurtosis: 20.410869598388672
	20th percentile R2(t,t-1): 0.908935010433197
	taking step 4
	median kurtosis: 13.953994750976562
	20th percentile R2(t,t-1): 0.9450386166572571
	taking step 5
	median kurtosis: 9.980635643005371
	20th percentile R2(t,t-1): 0.963342547416687
	taking step 6
	median kurtosis: 7.617109298706055
	20th percentile R2(t,t-1): 0.9739270806312561
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
4
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.007599240075992401 , 4 PCs used
total r^2 between top 4 NAM PCs and outcome is 0.13


In [12]:
res = cna_test(meta,harmony, umap, 'Final_Activity', 
               covars = ['Pred_use', 'Final_ISN_[IV]',
                         'Final_ISN_[IV][V]',
                         'Final_ISN_[V]',
                         'Final_Site_JHU',
                         'Final_Site_NYU'])

(140034, 39)
(140034, 20)
(140034, 2)
['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


computing default knn graph
qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 46.349185943603516
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 30.39740753173828
	20th percentile R2(t,t-1): 0.7639257311820984
	taking step 3
	median kurtosis: 20.410869598388672
	20th percentile R2(t,t-1): 0.908935010433197
	taking step 4
	median kurtosis: 13.953994750976562
	20th percentile R2(t,t-1): 0.9450386166572571
	taking step 5
	median kurtosis: 9.980635643005371
	20th percentile R2(t,t-1): 0.963342547416687
	taking step 6
	median kurtosis: 7.617109298706055
	20th percentile R2(t,t-1): 0.9739270806312561
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.22147785221477853 , 1 PCs used
total r^2 between top 1 NAM PCs and outcome is 0.03


## Single Nuclear

In [127]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/activity/sn_meta.csv')
print(meta.shape)

(40416, 11)


In [128]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/activity/sn_harmony.csv')
print(harmony.shape)

(40416, 20)


In [129]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/activity/sn_umap.csv')
print(umap.shape)

(40416, 2)


In [130]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Final_Chronicity', 'Final_Activity'])
d.samplem.head()

['cell' 'Sex' 'sample' 'cluster_names' 'Responder.Status' 'Race'
 'Final_ISN' 'Type']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


,Final_Chronicity,Final_Activity
sample,,
AMPSLEkid_cells_0137,3.0,5.0
AMPSLEkid_cells_0138,0.0,0.0
AMPSLEkid_cells_0139,3.0,2.0
AMPSLEkid_cells_0147,9.0,1.0
AMPSLEkid_cells_0366,7.0,2.0


In [131]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [132]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [133]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Activity, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 11.842386245727539
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 8.491312503814697
	20th percentile R2(t,t-1): 0.786330497264862
	taking step 3
	median kurtosis: 6.386176347732544
	20th percentile R2(t,t-1): 0.9200137615203857
stopping after 3 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing neighborhood-level FDRs
17
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.36136386361363865 , 17 PCs used
total r^2 between top 17 NAM PCs and outcome is 0.61


In [135]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Activity, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      covs = d.samplem[['Final_Chronicity']], 
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2)) 

covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing neighborhood-level FDRs
16
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.47535246475352466 , 16 PCs used
total r^2 between top 16 NAM PCs and outcome is 0.57


# ISN

## Single Cell

In [13]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/ISN/sc_meta.csv')
print(meta.shape)

(154032, 39)


In [14]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/ISN/sc_harmony.csv')
print(harmony.shape)

(154032, 20)


In [15]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/ISN/sc_umap.csv')
print(umap.shape)

(154032, 2)


In [ ]:
res = cna_test(meta, harmony, umap, 'Final_ISN_[IV]')

(154032, 39)
(154032, 20)
(154032, 2)
['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies
computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 50.24106979370117
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 33.02307891845703
	20th percentile R2(t,t-1): 0.7615886211395264
	taking step 3
	median kurtosis: 22.02227783203125
	20th percentile R2(t,t-1): 0.9084737420082092
	taking step 4
	median kurtosis: 14.879904747009277
	20th percentile R2(t,t-1): 0.9440323233604431
	taking step 5
	median kurtosis: 10.545235633850098
	20th percentile R2(t,t-1): 0.9626616954803466
	taking step 6
	median kurtosis: 7.996293544769287
	20th percentile R2(t,t-1): 0.9743828177452087
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
4
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.025597440255974404 , 4 PCs used
total r^2 between top 4 NAM PCs and outcome is 0.10


In [21]:
res = cna_test(meta, harmony, umap, 'Final_ISN_[IV]', 
               covars = ['Final_Activity'])

(154032, 39)
(154032, 20)
(154032, 2)
['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


computing default knn graph
qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 50.24106979370117
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 33.02307891845703
	20th percentile R2(t,t-1): 0.7615886211395264
	taking step 3
	median kurtosis: 22.02227783203125
	20th percentile R2(t,t-1): 0.9084737420082092
	taking step 4
	median kurtosis: 14.879904747009277
	20th percentile R2(t,t-1): 0.9440323233604431
	taking step 5
	median kurtosis: 10.545235633850098
	20th percentile R2(t,t-1): 0.9626616954803466
	taking step 6
	median kurtosis: 7.996293544769287
	20th percentile R2(t,t-1): 0.9743828177452087
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
8
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.278972102789721 , 8 PCs used
total r^2 between top 8 NAM PCs and outcome is 0.10


In [20]:
res = cna_test(meta, harmony, umap, 'Final_ISN_[V]')

(154032, 39)
(154032, 20)
(154032, 2)


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies
computing default knn graph
qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 50.24106979370117
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 33.02307891845703
	20th percentile R2(t,t-1): 0.7615886211395264
	taking step 3
	median kurtosis: 22.02227783203125
	20th percentile R2(t,t-1): 0.9084737420082092
	taking step 4
	median kurtosis: 14.879904747009277
	20th percentile R2(t,t-1): 0.9440323233604431
	taking step 5
	median kurtosis: 10.545235633850098
	20th percentile R2(t,t-1): 0.9626616954803466
	taking step 6
	median kurtosis: 7.996293544769287
	20th percentile R2(t,t-1): 0.9743828177452087
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
3
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.023797620237976203 , 3 PCs used
total r^2 between top 3 NAM PCs and outcome is 0.09


In [22]:
res = cna_test(meta, harmony, umap, 'Final_ISN_[V]', 
               covars = ['Final_Activity', 
                         'Pred_use',
                         'Race_[A]',
                         'Race_[B]',
                         'Race_[W]',
                         'First_biop',
                         'Final_Site_JHU',
                         'Final_Site_NYU'])

(154032, 39)
(154032, 20)
(154032, 2)
['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


computing default knn graph
qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 50.24106979370117
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 33.02307891845703
	20th percentile R2(t,t-1): 0.7615886211395264
	taking step 3
	median kurtosis: 22.02227783203125
	20th percentile R2(t,t-1): 0.9084737420082092
	taking step 4
	median kurtosis: 14.879904747009277
	20th percentile R2(t,t-1): 0.9440323233604431
	taking step 5
	median kurtosis: 10.545235633850098
	20th percentile R2(t,t-1): 0.9626616954803466
	taking step 6
	median kurtosis: 7.996293544769287
	20th percentile R2(t,t-1): 0.9743828177452087
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.1787821217878212 , 10 PCs used
total r^2 between top 10 NAM PCs and outcome is 0.13


## Single Nuclear

In [152]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/ISN/sn_meta.csv')
print(meta.shape)

(50173, 27)


In [153]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/ISN/sn_harmony.csv')
print(harmony.shape)

(50173, 20)


In [154]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/ISN/sn_umap.csv')
print(umap.shape)

(50173, 2)


In [99]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[A][B]', 'Race_[B]', 'Race_[B][AI]', 'Race_[B][W]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 'Final_Site_Cincinnati',
       'Final_Site_Einstein', 'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 'Final_Site_UCLA',
       'Final_Site_UCSD', 'Final_Site_UCSF'])
d.samplem.head()

/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


['cell' 'sample' 'cluster_names' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[A][B],Race_[B],...,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCLA,Final_Site_UCSD,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0142,1.0,1.489979,4.0,16.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0366,2.0,-1.068691,7.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
AMPSLEkid_cells_0966,1.0,-0.794548,5.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
AMPSLEkid_cells_2748,1.0,-1.068691,8.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0138,1.0,0.576169,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [100]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [101]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [158]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Final_ISN_[IV]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 13.259119033813477
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 9.410001754760742
	20th percentile R2(t,t-1): 0.7799340009689331
	taking step 3
	median kurtosis: 6.884187698364258
	20th percentile R2(t,t-1): 0.9179095029830933
stopping after 3 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing neighborhood-level FDRs
18
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.9583041695830417 , 18 PCs used
total r^2 between top 18 NAM PCs and outcome is 0.34


In [159]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Final_ISN_[V]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
12
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.7552244775522448 , 12 PCs used
total r^2 between top 12 NAM PCs and outcome is 0.34


In [114]:
pd.DataFrame.from_dict(sc_uni, orient='index').T.to_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/univariate_cna.csv')

In [83]:
sc_cond['Sex'] = "NA"
sc_cond['Responder_Status'] = "NA"
sc_cond['Prednisone_use'] = "NA"
sc_cond['Age'] = "NA"
pd.DataFrame.from_dict(sc_cond, orient='index').T.to_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/conditional_cna.csv')

In [30]:
sc_chron['Sex'] = "NA"
sc_chron['Responder_Status'] = "NA"
sc_chron['Prednisone_use'] = "NA"
sc_chron['Age'] = "NA"
sc_chron['Activity'] = "NA"
sc_chron['ISN'] = "NA"
sc_chron['Site'] = "NA"
sc_chron['Chronicity'] = "NA"
sc_chron['First_biopsy'] = "NA"
pd.DataFrame.from_dict(sc_chron, orient='index').T.to_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/PT/conditional_chronicity_cna.csv')